In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.transforms import ToTensor, Normalize,Compose,Resize
from torchvision.transforms.functional import to_pil_image
from torchvision.datasets import DatasetFolder
from sklearn.model_selection import KFold
from unet import UNet


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

# 自定义数据集类
class SegmentationDataset(DatasetFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super(SegmentationDataset, self).__init__(root, loader=lambda x: x, extensions=('jpg', 'png'),
                                                  transform=transform, target_transform=target_transform)

    def __getitem__(self, index):
        print(type(image))
        print(type(mask))
        image = self.loader(self.samples[index][0])
        mask = self.loader(self.samples[index][0].replace('images', 'masks'))
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            mask = self.target_transform(mask)
        return image, mask

# 定义数据集路径和预处理操作
data_root = '/workspace/Unet/bad/'
transform = Compose([
    Resize((256, 256)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:

# 创建数据集实例
dataset = SegmentationDataset(data_root, transform=transform)

# 定义 k-fold 交叉验证
k = 4
kf = KFold(n_splits=k, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
print(dataset)

Dataset SegmentationDataset
    Number of datapoints: 400
    Root location: /workspace/Unet/bad/
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )


In [5]:
for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):
    print(f'Fold: {fold+1}')

    # 创建训练集和验证集的数据加载器
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    # 定义模型
    model = UNet(n_channels=3,n_classes=2)
    model.to(device)

    # 定义损失函数
    criterion = nn.BCELoss()

    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 训练模型
    num_epochs = 10

    for epoch in range(num_epochs):
        running_loss = 0.0

        # 训练
        model.train()
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss}')

        # 验证
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)

                outputs = model(images)
                loss = criterion(outputs, masks)

                val_loss += loss.item()

            val_loss /= len(val_loader)
            print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}')

    # 保存模型
    torch.save(model.state_dict(), f'segmentation_model_fold{fold+1}.pth')


Fold: 1


TypeError: img should be PIL Image. Got <class 'str'>